# 샘플

In [ ]:
# 탭 나누는 샘플코드
import gradio as gr

def process_text1(input_text):
    return f"Tab 1에서 입력된 텍스트: {input_text}"

def process_text2(input_text):
    return f"Tab 2에서 입력된 텍스트: {input_text}"

# 각 탭을 구성하는 인터페이스 목록
tab1 = gr.Interface(
    fn=process_text1, 
    inputs=gr.Textbox(placeholder="Tab 1에 텍스트 입력", label="Tab 1 입력"), 
    outputs=gr.Textbox())
tab2 = gr.Interface(
    fn=process_text2, 
    inputs=gr.Textbox(placeholder="Tab 2에 텍스트 입력", label="Tab 2 입력"), 
    outputs=gr.Textbox())

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.TabbedInterface(interface_list=[tab1, tab2])

demo.launch()

# 프로젝트 코드

In [ ]:
import gradio as gr
import ollama
from langchain.document_loaders import TextLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS



# 텍스트 파일을 읽어들이는 함수
def read_txt(file):
    # Gradio의 File 객체는 file.read()로 바이너리 데이터를 반환하므로, 이를 디코딩하여 텍스트로 변환
    return file.read().decode('utf-8')

# 텍스트 파일을 학습시키고 벡터화하는 함수
def process_text(file):
    # 텍스트 파일 내용 읽기
    file_content = read_txt(file)
    
    # 문서 로더를 사용해 텍스트를 문서로 변환
    loader = TextLoader(file.name)
    documents = loader.load()

    # SentenceTransformer를 사용해 텍스트를 임베딩
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    
    # FAISS를 사용하여 벡터를 저장
    vector_store = FAISS.from_documents(documents, embeddings)
    
    # 벡터스토어 반환
    return vector_store

# Ollama 모델을 사용하여 텍스트 기반 질문에 답하는 함수
def answer_question(file, question):
    # 텍스트 파일 처리 및 벡터화
    vector_store = process_text(file)
    
    # FAISS 벡터스토어에서 가장 관련성이 높은 문서를 검색
    docs = vector_store.similarity_search(question, k=3)
    
    # Ollama 모델을 사용하여 질문에 대한 답변 생성
    prompt = f"다음 텍스트를 기반으로 질문에 답변하세요:\n{docs}\n질문: {question}\n답변:"

    try:
        # Ollama 모델을 사용하여 답변 생성
        response = ollama.chat(model="llama2", messages=[{"role": "user", "content": prompt}])
        return response['text']
    except Exception as e:
        return f"오류 발생: {str(e)}"

# Gradio 인터페이스 설정
def create_gradio_interface():
    # 텍스트 파일 업로드 입력
    file_input = gr.File(label="Upload a Text File")
    
    # 질문 입력 및 답변 출력
    question_input = gr.Textbox(label="Ask a Question", placeholder="Enter your question here")
    question_output = gr.Textbox(label="Answer")
    
    # Gradio 인터페이스 설정
    with gr.Blocks() as iface:
        gr.Interface(fn=answer_question, inputs=[file_input, question_input], outputs=question_output).launch()

    return iface

# Gradio 인터페이스 실행
if __name__ == "__main__":
    iface = create_gradio_interface()
    iface.launch()

d:\241223\aiProject2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [5]:
iface.close();

Closing server running on port: 7862


In [ ]:
import gradio as gr
from langchain.document_loaders import TextLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.llms import Ollama
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager

class show_recipe:
    def __init__(self):
        self.answer_template = """
        당신은 음식에 대한 조리법을 제공하는 AI 어시스턴트입니다.
        다음 정보를 바탕으로 사용자의 질문에 대해 한국어로 답변해주세요:
        
        아래는 이전 대화 내용입니다:
        {chat_history}

        관련 문서 내용:
        {context}
        
        사용자 질문: {question}
        
        규칙:
        1. 결과를 자연스러운 한국어로 설명해주세요
        2. 모든 답변은 친절하고 전문적으로 제공해주세요.
        3. 결과가 없다면 그 이유를 설명해주세요
        4. 요리가 익숙하지 않은 사람도 이해하기 쉽게 설명해주세요
        5. 조리방법을 번호를 매겨 순서대로 설명해주세요
        6. 
        """









        # Gemma2 모델 초기화
        callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        self.llm = Ollama(
            model="gemma2",
            temperature=0,
            callback_manager=callback_manager
        )

        # 프롬프트 템플릿 설정
        self.query_prompt = ChatPromptTemplate.from_template(self.query_template)
        self.answer_prompt = ChatPromptTemplate.from_template(self.answer_template)

        # Chain 설정
        self.query_chain = LLMChain(llm=self.llm, prompt=self.query_prompt)
        self.answer_chain = LLMChain(llm=self.llm, prompt=self.answer_prompt)

    def generate_query(self, question: str, schema_info: str, feedback_info: str = "") -> str:
        """질문에 대한 SQL 쿼리를 생성합니다."""
        response = self.query_chain.run(
            question=question,
            schema_info=schema_info,
            feedback_info=feedback_info
        )
        return self.extract_sql_query(response)

    def generate_answer(self, question: str, query: str, result: Any) -> str:
        """쿼리 결과를 바탕으로 자연어 답변을 생성합니다."""
        result_str = str(result) if isinstance(result, pd.DataFrame) else json.dumps(result, ensure_ascii=False)
        response = self.answer_chain.run(
            question=question,
            query=query,
            result=result_str
        )
        return response.strip()